[Aufgaben](merkmale_aufgaben.ipynb)

In [ ]:
%matplotlib inline
%pylab inline
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import scipy.constants as const
import scipy
from scipy.io import wavfile
from pylab import *
from scipy.fftpack import fft, ifft
from IPython.display import Audio

In [ ]:
import functions.timedomain as td

#### <font color="#FF0000">Eingabezelle</font>

In [ ]:
filename = '72'
zf_len = 0.025
zf_shape = 'hamm'
overlap = 0.9
td_data,fs = td.show_data(filename)

In [ ]:
Audio(data = td_data, rate = fs) 

#### <font color="#FF0000">Eingabezelle</font>

In [ ]:
td.st_energy(td_data, fs, zf_len, zf_shape, overlap)

In [ ]:
##### zf = 0.025   # Zeitfensterlänge
offs = 0.01  # Zeitfensterversatz
start_zf = 0.3
atw_zf1 = int(start_zf * fs)    #Start ZF1
atw_zf2 = atw_zf1 + 441    #Start ZF2
#atw_zf3 = 18000    #Start ZF3

In [ ]:
t = np.linspace(0, (n + n1)/fs, n+n1)
plt.figure(figsize=(10,8))
plt.subplot(311)
n = int(round(zf * fs))
n1 = int(round(offs * fs))
title_name = '1. Zeitfenster'
plt.title(title_name)
res = np.zeros(n + n1)
res[0:n] = numarray[atw_zf1:atw_zf1+n]
plt.plot(t, res)
plt.axis([0, 0.035, min(res),max(res)])
plt.grid()
plt.subplot(312)
title_name = '2. Zeitfenster'
plt.title(title_name)
res = np.zeros(n + n1)
res[n1:n1+n] = numarray[atw_zf2:atw_zf2+n]
plt.subplots_adjust(hspace = 0.5)
plt.axis([0, 0.035, min(res),max(res)])
plt.grid()
plt.xlabel('t in s')
plt.plot(t, res)

#### Zeitfensterbewertung 

#### <font color="#FF0000">Eingabezelle</font>

In [ ]:
#window = hanning(n)
#wform = 'Hanning'
window = hamming(n)
wform = 'Hamming'
#window = bartlett(n)
#wform = 'Bartlett'

In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(311)
title_name = '1. Zeitfenster (bewertet)'
plt.title(title_name)
wdat1 = window * numarray[atw_zf1:atw_zf1+n]
wdat2 = window * numarray[atw_zf2:atw_zf2+n]
res = np.zeros(n + n1)
res[0:n] = wdat1
plt.axis([0,0.035,min(res),max(res)])
plt.grid()
plt.plot(t, res)
res = np.zeros(n + n1)
res[n1:n1+n] = wdat2
plt.subplot(312)
plt.subplots_adjust(hspace = 0.5)
title_name = '2. Zeitfenster (bewertet)'
plt.title(title_name)
plt.axis([0,0.035,min(res),max(res)])
plt.grid()
plt.plot(t, res)

### Energieberechnung für jedes Zeitfenster

In [ ]:
anz_zf = int(len(numarray)//n1 - 2)
zf_e = np.zeros(anz_zf)
for i in np.arange(anz_zf):
    zf_e[i] = np.sum((numarray[int(i*n1):int(i*n1 + n)]* window)**2)
plt.figure(figsize=(10,4))
plt.title('Energieverlauf des Signals')
t_e = np.linspace(0, len(numarray)/fs, anz_zf)
plt.xlabel('t in s')
plt.grid()
plt.plot(t_e, zf_e)

### Berechnung Autokorrelation
bewertetes Zeitfenster

In [ ]:
def autocorr(x):
    result = numpy.correlate(x, x, mode='full')
    return result[int(result.size/2):]

t = np.linspace(0, n/fs, n)
auto_k = autocorr(np.append(wdat1, wdat1))
plt.figure(figsize=(10,4))
plt.title('Autokorrelation eines Zeitfensters')
plt.axis([0, 0.025, min(auto_k), max(auto_k)])
plt.xlabel('Verschiebung k in s')
plt.grid()
plt.plot(t, auto_k[0:int(len(auto_k)/2)])